In [0]:
### CODE TO ACCESS DRIVE ###
# Run
# Follow Link
# Copy Link, Paste and Enter 
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Change to Drive folder - might be different path
%cd /content/drive/My Drive/'Colab Notebooks'/
#%cd /content/drive/

/content/drive/My Drive/Colab Notebooks


In [0]:
# Check in correct folder 
%ls

 checkpoint.pkl                             logs/
'Copy of deep_learning_project (1).ipynb'   MCcheckpoint.pkl
'Copy of deep_learning_project.ipynb'       MC_scores.p
 dataset1.py                                MLMCcheckpoint.pkl
 dataset.py                                 MLMC_scores.p
 deep_learning_project.ipynb                pictures/
 Lab4.ipynb                                 __pycache__/
 labels.p                                   UrbanSound8K_test.pkl
 LMCcheckpoint.pkl                          UrbanSound8K_train.pkl
 LMC_scores.p


In [0]:
## Libraries
import time
from multiprocessing import cpu_count
from typing import Union, NamedTuple

import torch
import torch.backends.cudnn
import numpy as np
from torch import nn, optim
from torch.nn import functional as F
import torchvision.datasets
from torch.optim.optimizer import Optimizer
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

import argparse
import os
from pathlib import Path

from torch.utils import data
from dataset import UrbanSound8KDataset
import random
import pickle 

torch.backends.cudnn.benchmark = True
parser = argparse.ArgumentParser(
                        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
                        )
## Arguments
default_dataset_dir = Path.home() / ".cache" / "torch" / "datasets"
parser.add_argument("--log-dir", default=Path("logsold"), type=Path)
parser.add_argument("--dataset-root", default=default_dataset_dir)
parser.add_argument("--learning_rate",default = 1e-3, type=float, help="Learning rate")
parser.add_argument("--sgd_momentum",default =  0.9, type=float)
parser.add_argument("--dropout", default = 0.5, type = float)
parser.add_argument("--score_calc", default = True, type = bool, help ="whether to print out currrent scores of models")
parser.add_argument("--mode", default = 'TSCNN', type = str, help="Mode of network to train. Run both LMC and MC before TSCNN. Options: LMC, MC, MLMC, TSCNN, SCORES")
parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")
parser.add_argument(
    "--batch-size",
    default=32,
    type=int,
    help="Number of images within each mini-batch",
)
parser.add_argument(
    "--epochs",
    default=31,
    type=int,
    help="Number of epochs (passes through the entire dataset) to train for",
)
parser.add_argument(
    "--val-frequency",
    default=1,
    type=int,
    help="How frequently to test the model on the validation set in number of epochs",
    )
parser.add_argument(
    "--log-frequency",
    default=10,
    type=int,
    help="How frequently to save logs to tensorboard in number of steps",
)
parser.add_argument(
    "--print-frequency",
    default=100,
    type=int,
    help="How frequently to print progress to the command line in number of steps",
)
parser.add_argument(
    "-j",
    "--worker-count",
    default=cpu_count(),
    type=int,
    help="Number of worker processes used to load data.",
)
parser.add_argument(
    "--checkpoint-path",
    default="checkpointold.pkl",
    type=str,
    help="Provide a file to store checkpoints of the model parameters during training."
)
parser.add_argument(
    "--weight_decay",
    default=1e-5,
    type=float,
    help="Weight decay: parameter related to L-2 regularisation.",
)

class ImageShape(NamedTuple):
    height: int
    width: int
    channels: int

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

def main(args):
    args.dataset_root.mkdir(parents=True, exist_ok=True)

    ## specifiy checkpoint path
    mode = args.mode
    checkpoint_path = args.checkpoint_path
    checkpoint_path = Path(mode + checkpoint_path)

    if args.score_calc == True: 
        modes = ['LMC', 'MC', 'MLMC']
        class_labels = ["ac", "ch", "cp", "db", "dr", "ei", "gs", "jh", "si", "sm"]
        labels = pickle.load(open('labels.p', 'rb'))
        # load saved scores
        for i in modes: 
            scores = pickle.load(open(f"{i}_scores.p", "rb"))
            preds = scores.argmax(dim=-1).cpu().numpy()
            accuracy, per_class_accuracy = compute_accuracy(
                np.array(preds), np.array(labels)
            )
            print(f"{i}_accuracy: {accuracy * 100:2.1f}")
            for j in range(0,10):
                print(f"{class_labels[j]}_acc: {per_class_accuracy[j] * 100:2.1f}")

    elif mode == 'TSCNN':
        LMC_scores = pickle.load(open('LMC_scores.p', 'rb'))
        MC_scores = pickle.load(open('MC_scores.p', 'rb'))
        labels = pickle.load(open('labels.p', 'rb'))

        scores = LMC_scores + MC_scores
        
        preds = scores.argmax(dim=-1).cpu().numpy()
        accuracy, per_class_accuracy = compute_accuracy(
              np.array(preds), np.array(labels)
          )
        class_labels = ["ac", "ch", "cp", "db", "dr", "ei", "gs", "jh", "si", "sm"]
        print(f"accuracy: {accuracy * 100:2.1f}")
        for i in range(0,10):
            print(f"{class_labels[i]}_acc: {per_class_accuracy[i] * 100:2.1f}")

    else: 
        ## load data
        train_loader = torch.utils.data.DataLoader(
            UrbanSound8KDataset('UrbanSound8K_train.pkl', mode),
              batch_size=args.batch_size, shuffle=True,
              num_workers=args.worker_count, pin_memory=True
              )

        val_loader = torch.utils.data.DataLoader(
            UrbanSound8KDataset('UrbanSound8K_test.pkl', mode),
              batch_size=1, shuffle=False,
              num_workers=args.worker_count, pin_memory=True)
        
        height_val = 85
        if mode == 'MLMC': 
            height_val = 145
        model = CNN(height=height_val, width=41, channels=1, class_count=10, dropout = args.dropout, mode = args.mode)
        criterion = lambda logits, labels : nn.CrossEntropyLoss()(logits, labels)
        optimizer = torch.optim.Adam(model.parameters(), args.learning_rate)#, weight_decay=args.weight_decay)#, momentum =args.sgd_momentum)


        log_dir = get_summary_writer_log_dir(args)
        print(f"Writing logs to {log_dir}")
        summary_writer = SummaryWriter(
                str(log_dir),
                flush_secs=5
        )
        trainer = Trainer(
            model, train_loader, val_loader, criterion, optimizer, summary_writer, DEVICE, 
            checkpoint_path
        )
        trainer.train(
            args.epochs,
            args.val_frequency,
            print_frequency=args.print_frequency,
            log_frequency=args.log_frequency,
        )

        summary_writer.close()

        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model'])
        validation = Trainer(model, train_loader, val_loader, criterion, optimizer, summary_writer, DEVICE, 
            checkpoint_path)
        dummy, scores, labels = validation.validate(100, 1, 1)
        scores = scores.cpu()
        pickle.dump(scores, open(f"{mode}_scores.p", "wb"))
        pickle.dump(labels, open("labels.p", "wb"))
    

class CNN(nn.Module):
    def __init__(self, height: int, width: int, channels: int, class_count: int, dropout: float, mode: str):
        super().__init__()
        self.input_shape = ImageShape(height=height, width=width, channels=channels)
        self.class_count = class_count
        ## Convolution layer 1
        self.conv1 = nn.Conv2d(
            in_channels=self.input_shape.channels,
            out_channels=32,
            kernel_size=(3,3),
            padding=(1,1),
            stride=(1,1),
        )
        # batch normalisation
        self.conv1_BN = nn.BatchNorm2d(32)
        self.initialise_layer(self.conv1)

        ## Convolution layer 2
        self.conv2 = nn.Conv2d(
            in_channels=self.conv1.out_channels,
            out_channels=32,
            kernel_size=(3,3),
            padding=(1,1),
            stride=(1,1),
            )
        # dropout
        self.dropout2d = nn.Dropout2d(dropout)
        self.conv2_BN = nn.BatchNorm2d(32)
        self.initialise_layer(self.conv2)
        # pooling layer
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2), padding=(1,1))

        ## Convolution layer 3
        self.conv3 = nn.Conv2d(
            in_channels=self.conv2.out_channels,
            out_channels=64,
            kernel_size=(3,3),
            padding=(1,1),
            stride=(1,1),
        )
         
        self.conv3_BN = nn.BatchNorm2d(64)
        self.initialise_layer(self.conv3)
        ## Convolution layer 4
        self.conv4 = nn.Conv2d(
            in_channels=self.conv3.out_channels,
            out_channels=64,
            kernel_size=(3,3),
            padding=(1,1),
            stride=(1,1),
            )
        self.conv4_BN = nn.BatchNorm2d(64)
        self.initialise_layer(self.conv4)

        ## Fully-Connected layer
        in_size = 15488
        if mode == 'MLMC':
            in_size = 26048
        self.fc1 = nn.Linear(in_size, 1024)
        self.fc1_BN = nn.BatchNorm1d(1024)
        self.dropout = nn.Dropout(dropout)
        self.initialise_layer(self.fc1)
        
        ## Output layer
        self.fc3 = nn.Linear(1024, 10)
        self.initialise_layer(self.fc3)

    def forward(self, images: torch.Tensor) -> torch.Tensor:
        # Conv 1 --> batch norm --> relu
        x = F.relu(self.conv1_BN(self.conv1(images)))
        # Conv 2 --> batch norm --> relu --> dropout --> pooling
        x = self.dropout2d(F.relu(self.conv2_BN(self.conv2(x))))
        x = self.pool(x)
        # Conv 3 --> batch norm --> relu --> dropout
        x = self.dropout2d(F.relu(self.conv3_BN(self.conv3(x))))
        # Conv 4 --> batch norm --> relu --> dropout --> pooling
        x = self.dropout2d(F.relu(self.conv4_BN(self.conv4(x))))
        x = self.pool(x)
        # Flatten output of pooling layer
        x = torch.flatten(x, 1)
        # FC layer 1 --> batch norm --> sigmoid --> dropout
        x = self.dropout(torch.sigmoid(self.fc1_BN(self.fc1(x))))
        # Output layer
        x = self.fc3(x)
        return x

    @staticmethod
    def initialise_layer(layer):
        if hasattr(layer, "bias"):
            nn.init.zeros_(layer.bias)
        if hasattr(layer, "weight"):
            nn.init.kaiming_normal_(layer.weight)


class Trainer:
    def __init__(
        self,
        model: nn.Module,
        train_loader: DataLoader,
        val_loader: DataLoader,
        criterion: nn.Module,
        optimizer: Optimizer,
        summary_writer: SummaryWriter,
        device: torch.device,
        checkpoint_path: Path,
    ):
        self.model = model.to(device)
        self.device = device
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.summary_writer = summary_writer
        self.step = 0
        self.checkpoint_path = checkpoint_path

    def train(
        self,
        epochs: int,
        val_frequency: int,
        print_frequency: int = 20,
        log_frequency: int = 5,
        start_epoch: int = 0
    ):
        self.model.train()
        best_acc = 0
        for epoch in range(start_epoch, epochs):
            self.model.train()
            data_load_start_time = time.time()
            for batch, labels, filename in self.train_loader:
                batch = batch.to(self.device)
                labels = labels.to(self.device)
                data_load_end_time = time.time()
                # Forward pass
                logits = self.model.forward(batch)
                # Compute loss
                loss = self.criterion(logits, labels)
                # Compute the backward pass
                loss.backward()
                # Step optimizer
                self.optimizer.step()
                # Zero gradient buffers
                self.optimizer.zero_grad()
                # Compute accuracy
                with torch.no_grad():
                    preds = logits.argmax(-1)
                    accuracy, per_class_accuracy = compute_accuracy(labels, preds)

                # output metrics and log files 
                data_load_time = data_load_end_time - data_load_start_time
                step_time = time.time() - data_load_end_time
                if ((self.step + 1) % log_frequency) == 0:
                    self.log_metrics(epoch, accuracy, loss, data_load_time, step_time)
                if ((self.step + 1) % print_frequency) == 0:
                    self.print_metrics(epoch, accuracy, loss, data_load_time, step_time)

                self.step += 1
                data_load_start_time = time.time()
 
            self.summary_writer.add_scalar("epoch", epoch, self.step)

            if ((epoch + 1) % val_frequency) == 0:
                # switch to validation mode
                best_acc, dummy, dummy1 = self.validate(best_acc, epoch, epochs) #, ~
                # switch back to train mode
                self.model.train()
    def print_metrics(self, epoch, accuracy, loss, data_load_time, step_time):
        # online print meterics 
        epoch_step = self.step % len(self.train_loader)
        print(
                f"epoch: [{epoch}], "
                f"step: [{epoch_step}/{len(self.train_loader)}], "
                f"batch loss: {loss:.5f}, "
                f"batch accuracy: {accuracy * 100:2.2f}, "
                f"data load time: "
                f"{data_load_time:.5f}, "
                f"step time: {step_time:.5f}"
        )

    def log_metrics(self, epoch, accuracy, loss, data_load_time, step_time):
        # summary writer for logs files
        self.summary_writer.add_scalar("epoch", epoch, self.step)
        self.summary_writer.add_scalars(
                "accuracy",
                {"train": accuracy},
                self.step
        )
        self.summary_writer.add_scalars(
                "loss",
                {"train": float(loss.item())},
                self.step
        )
        self.summary_writer.add_scalar(
                "time/data", data_load_time, self.step
        )
        self.summary_writer.add_scalar(
                "time/data", step_time, self.step
        )

    def validate(self, best_acc, epoch, epochs):
        '''Model validation - validates the model by computing per file accuracy. 

        Args: 
            best_acc: (float) current best accuracy for the run
            epoch: (int) current epoch No. 
            epochs: (int) total number of epochs

        Returns: 
            best_acc: (float) current best accuracy for the run
            save_scores: (tensor) contains all per file averages scores for use in TSCNN model 
        ''' 
        results = {"preds": [], "labels": []}
        total_loss = 0
        self.model.eval()
        current_filename = ''
        file_scores = torch.zeros([1,10])
        file_scores = file_scores.to(self.device)
        save_scores = torch.zeros([836, 10])
        save_scores = save_scores.to(self.device)
        n = 0 

        # No need to track gradients for validation, we're not optimizing.
        with torch.no_grad():
            for batch, labels, filename in self.val_loader:
                # all files are in order, as soon as new filename detected -> compute prediction 
                if current_filename != filename and current_filename != '':
                    # softmax for normalisation 
                    file_scores = F.softmax(file_scores, 1)
                    # take the average class-wise
                    file_scores = torch.sum(file_scores, 0)/(file_scores.size()[0]-1)
                    # calc prediction 
                    preds = file_scores.argmax(dim=-1).cpu().numpy()
                    results["preds"].extend([preds])
                    results["labels"].extend(list(current_label.cpu().numpy()))
                    # save scores for end validation scores  
                    save_scores[n] = file_scores
                    n += 1
                    file_scores = torch.zeros([1,10])
                    file_scores = file_scores.to(self.device)
                
                # calc logits 
                batch = batch.to(self.device)
                labels = labels.to(self.device)
                logits = self.model(batch)
                loss = self.criterion(logits, labels)
                total_loss += loss.item()
                # concat scores while same filename 
                file_scores = torch.cat((file_scores, logits), 0)
                # save filename to compare against next
                current_filename = filename
                current_label = labels

        # compute validation accuracy 
        accuracy, per_class_accuracy = compute_accuracy(
            np.array(results["labels"]), np.array(results["preds"])
        )
        # save model checkpoint if better average accuracy 
        if best_acc < accuracy:
                self.model_checkpoint(accuracy, epoch)
                best_acc = accuracy
                
        average_loss = total_loss / len(self.val_loader)
        
        # summary writer statistics 
        self.summary_writer.add_scalars(
                "accuracy",
                {"test": accuracy},
                self.step
        )
        self.summary_writer.add_scalars(
                "loss",
                {"test": average_loss},
                self.step
        )
        
        # output accuracy, loss and class-wise accuracy 
        class_labels = ["ac", "ch", "cp", "db", "dr", "ei", "gs", "jh", "si", "sm"]
        print(f"validation loss: {average_loss:.5f}, accuracy: {accuracy * 100:2.2f}")
        for i in range(0,10):
            print(f"{class_labels[i]}_acc: {per_class_accuracy[i] * 100:2.2f}")

        return float(best_acc), save_scores, results['labels']

    def model_checkpoint(self, accuracy, epoch):
      """Saves model checkpoint under checkpoint path at the epoch. 

      Args: 
          accuracy: (float) current accuracy of model 
          epoch: (int) current epoch of model 
      """
      print(f"Saving model to {self.checkpoint_path}")
      torch.save({
          'epoch': epoch,
          'model': self.model.state_dict(),
          'accuracy': accuracy
      }, self.checkpoint_path)



def compute_accuracy(
    labels: Union[torch.Tensor, np.ndarray], preds: Union[torch.Tensor, np.ndarray]
) -> float:
    """Computes class-wise accuracy and accuracy and ensures no division by zero. 

    Args:
        labels: ``(batch_size, class_count)`` tensor or array containing example labels
        preds: ``(batch_size, class_count)`` tensor or array containing model prediction
    
    Returns: 
        Classwise accuracy, accuracy 
    """
    # check No. labels = No. preds
    assert len(labels) == len(preds)
    class_acc = torch.zeros(10)
    # computes class-wise accuracy 
    for i in range(0,10):
        c = (preds[labels == i] == i).sum()
        d = sum(labels == i)
        if d != 0:
            class_acc[i] = c/d
        elif c > 0:
            class_acc[i] = 0
        else:
            class_acc[i] = 100

    return float((labels == preds).sum()) / len(labels), class_acc


def get_summary_writer_log_dir(args: argparse.Namespace) -> str:
    """Get a unique directory that hasn't been logged to before for use with a TB
    SummaryWriter.

    Args:
        args: CLI Arguments

    Returns:
        Subdirectory of log_dir with unique subdirectory name to prevent multiple runs
        from getting logged to the same TB log directory (which you can't easily
        untangle in TB).
    """
    tb_log_dir_prefix = (
      f"CNN_bn_"
      f"run_"
      f"mode_{args.mode}"
    )
    i = 0
    while i < 1000:
        tb_log_dir = args.log_dir / (tb_log_dir_prefix + str(i))
        if not tb_log_dir.exists():
            return str(tb_log_dir)
        i += 1
    return str(tb_log_dir)


if __name__ == "__main__":
    main(parser.parse_args())

LMC_accuracy: 80.7
ac_acc: 78.4
ch_acc: 93.9
cp_acc: 62.9
db_acc: 78.0
dr_acc: 86.7
ei_acc: 83.1
gs_acc: 94.1
jh_acc: 89.3
si_acc: 82.5
sm_acc: 85.7
MC_accuracy: 79.9
ac_acc: 77.6
ch_acc: 92.6
cp_acc: 67.8
db_acc: 89.9
dr_acc: 76.7
ei_acc: 72.8
gs_acc: 84.2
jh_acc: 100.0
si_acc: 96.6
sm_acc: 69.1
MLMC_accuracy: 78.6
ac_acc: 82.9
ch_acc: 100.0
cp_acc: 52.8
db_acc: 80.8
dr_acc: 80.2
ei_acc: 90.5
gs_acc: 93.9
jh_acc: 94.1
si_acc: 68.1
sm_acc: 87.2
